(Enning, 04.01.2023)
## Wie orientiert man sich im Netz?

### Einführung in IP-Addressen, Routing und Firewalls

- Jeder Teilnehmer (Node) hat mindestens eine IP-Adresse
- Früher reichte der Addressraum von 4 Bytes vollkommen aus

In [1]:
f' mit 4 Bytes kann man {2**(4*8):,} Nodes eindeutige Adressen geben'

' mit 4 Bytes kann man 4,294,967,296 Nodes eindeutige Adressen geben'

- IPv4 Adressen werden in der Form `nn.nn.nn.nn` geschrieben, wobei `nn` jeweils die Dezimaldarstellung eines Bytes ist

### IPv4 Subnetze

- Subnetze:
  - Klasse A: Das erste Byte ist von [IANA](https://en.wikipedia.org/wiki/Internet_Assigned_Numbers_Authority) zugewiesen, die restlichen werden vom Administrator den Nodes zugewiesen
  - Klasse B: Die ersten zwei Bytes sind von IANA zugewiesen
  - Klasse C: Die ersten drei Bytes sind von IANA zugewiesen
- Alle Adressen können im Internet gerroutet werden. Ausnahmen Private Subnetze. Diese sind nicht eindeutig und werden nicht geroutet

### Private Subnetze
- In einem privaten Home- oder Wohnheimnetz haben Sie vermutlich eine IP-Adresse, die z.B. 192.168.100.20 lautet
- Adressen, die mit 192.168 beginnen kann es tausende Male weltweit geben
- In einem solchen Netz kann man 256 (-2 reservierte Adressen) Netzwerk-Nodes adressieren
- Diese können untereinander problemlos kommunizieren

Die von privaten Netzen verwendbaren Adressbereiche sind:

- Klasse A: 10.0.0.0 to 10.255.255.255.
- Klasse B: 172.16.0.0 to 172.31.255.255.
- Klasse C: 192.168.0.0 to 192.168.255.255.

### Domain Name System (DNS)

- das [Domain Name System](https://de.wikipedia.org/wiki/Domain_Name_System) (DNS) ist das verteilte "Telefonbuch" des Internet
- Host (Domain) Namen werden zu IP Adressen aufgelöst
- üblicherweise nutzt man zum Test Kommandozeilenprogramme wie `dig' 
- Es gibt aber auch ein Python-Package [dnspython](https://www.dnspython.org/)

### Address Resolution Protocol (ARP)

- DNS ist auf Schicht 3 und arbeitetet mit IP Adressen
- Für den Zugriff auf einen Adapter braucht man dessen [Media Access Control](https://de.wikipedia.org/wiki/MAC-Adresse) (MAC) Adresse 
- Diese Auflösung get mit dem [Address Resolution Protokoll](https://de.wikipedia.org/wiki/Address_Resolution_Protocol)

>__Hinweis__: DNS und ARP arbeiten in beide Richtungen, können also z.B. zu einer IP-Adresse des Hostnamen finden


### Routing
- In jedem Knoten wird hinterlegt, welchen Weg ein ausgehendes Datenpaket nehmen muss. Die Regeln dazu stehen in der Routingtabelle
- In einem "Router" wird nach Richtungen auf Adapter verteilt
- In einem "normalen" Node muss der Weg zum Router als "via" angegeben werden
- Auf Linux-Systemen lässt sie sich mit `ip route` abfragen.
>__Hinweis__: Auch unsere Docker-Container sind Linux-Systeme. Allerdings sind systemnahe Tools hier nicht installiert, damit Sie keinen Blödsinn machen

Dies ist die Routing-Tabelle meines PC's, wenn er per VPN mit der FH verbunden ist

    manfred@WAG-0C-37803997014:~$ ip route
    default via 192.168.178.1 dev enp3s0 proto dhcp metric 100 
    10.1.0.0/16 dev vpn0 proto static scope link metric 50 
    10.4.0.0/16 dev vpn0 proto static scope link metric 50 
    10.9.0.0/24 dev vpn0 proto static scope link metric 50 
    10.9.1.0/24 dev vpn0 proto static scope link metric 50 
    10.9.2.0/23 dev vpn0 proto static scope link metric 50 
    10.9.4.0/24 dev vpn0 proto static scope link metric 50 
    10.13.10.0/24 dev vpn0 proto static scope link metric 50 
    10.13.15.0/24 dev vpn0 proto static scope link metric 50 
    129.233.213.0/24 dev vpn0 proto static scope link metric 50 
    134.61.0.0/16 dev vpn0 proto static scope link metric 50 
    134.130.0.0/16 dev vpn0 proto static scope link metric 50 
    134.130.205.0/24 dev vpn0 proto static scope link metric 50 
    137.226.0.0/16 dev vpn0 proto static scope link metric 50 
    149.201.0.0/16 dev vpn0 proto static scope link metric 50 
    149.201.27.0/24 dev vpn0 proto kernel scope link src 149.201.27.147 metric 50 
    149.201.252.123 via 192.168.178.1 dev enp3s0 proto static metric 50 
    153.96.180.0/24 dev vpn0 proto static scope link metric 50 
    153.96.200.0/23 dev vpn0 proto static scope link metric 50 
    169.254.0.0/16 dev docker0 scope link metric 1000 linkdown 
    172.17.0.0/16 dev docker0 proto kernel scope link src 172.17.0.1 linkdown 
    172.18.0.0/16 dev br-fd498b3baa69 proto kernel scope link src 172.18.0.1 
    172.20.0.0/16 dev br-d4c9dd51a90a proto kernel scope link src 172.20.0.1 linkdown 
    192.35.229.0/24 dev vpn0 proto static scope link metric 50 
    192.44.11.0/24 dev vpn0 proto static scope link metric 50 
    192.102.148.0/24 dev vpn0 proto static scope link metric 50 
    192.168.178.0/24 dev enp3s0 proto kernel scope link src 192.168.178.23 metric 100 
    192.168.178.1 dev enp3s0 proto static scope link metric 50 

- Alles, was nicht woanders hin geschickt wird, geht auf die "Default-Route". 
- Viele Adressbereiche werden zum VPN-Adapter`vpn0` geschickt werden
- 10.1.0.0/16 ist ein Class A Subnetz. Also alle Adressen, die mit 10. beginnen gehen ins VPN
- 134.130.205.0/24 ist ein Class C Subnetz. Alle Adressen, die mit 134.130.205 beginnen gehen ins VPN

### Masquerading, Network Address Translation (NAT)
- Wenn man aus einem privaten Subnetz über einen "Router" ins Internet möchte, muss man "tricksen":
- Nach "innen" hat der Router eine Adresse im jeweiligen Subnetz
- Nach außen hat er (meistens noch) eine eigene, eindeutige IPv4 Adresse


### Firewall

- Um Netze gegen Angriffe von außen zu schützen sind meist nur wenige Transfers von außen nach innen gestattet
- Die Firewall ist der "Wächter", der dafür sorgt, dass nur zulässige Pakete durchgehen
- Die Firewall kümmert sich auch um das Masquerading
- Ein Paket das z.B. von 192.168.100.20 kommt und über einen Router ins Internet geroutet wird, muss nach außen so erscheinen, als käme es von Router (Nur er hat eine brauchbare Absenderadresse, nämlich seine externe IP)
- Es muss also das Source-Feld im IP-Paket "gefaked" werden (Network Address Translation, NAT, Masquerading)
- Antworten aus dem Internet müssen wieder an den richtigen internen Teilnehmer geroutet werden


### Dynamic Host Configuration Protocol (DHCP)

- In der Vergangenheit konnte ein Netzwerkadministrator die IP-Adressen verwalten und die Routing- und DNS Einstellungen der PCs vornehmen
- Heute: Es gibt zuwenig IPv4 Adressen, die werden nur noch "verliehen" (lease)
- Netzwerktopologien sind komplex und volatil
- Lösung Automatische Konfiguration der Clients

Ein DCHP-Server
- gibt IP Adressen aus einem vorgegebenen Adressraum aus
- gibt dem Client wichtige Informationen zu den Ressourcen des Netzwerks
    - wo ist der nächstgelegene DNS-Server zu finden?
    - welche Adresse hat der Router
    - unter welcher Adresse kann an alle gesendet werden (Broadcast)
    - unter welcher Adresse gibt es einen Zeitservice
    - (Wo ist der Anmeldeserver?)